In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=2b1cdb2003f84379218e7c7a0b4a4729a0f0443c8ff4a2ac65604505a0e6e5ce
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [2]:
import pyspark

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Assginment').getOrCreate()
spark

In [4]:
df=spark.read.csv('hdfs://localhost:9000/data/TweetsDataset.csv',header=True)
df.show(5)

+--------------------+--------+--------------------+--------+---------+
|  tweet_date_created|tweet_id|          tweet_text|language|sentiment|
+--------------------+--------+--------------------+--------+---------+
|2018-06-30T19:26:...|1.01E+18|I vote #BernardoC...|      en|  NEUTRAL|
|2018-08-12T14:23:...|1.03E+18|When is your firs...|      en|  NEUTRAL|
|2018-07-17T10:50:...|1.02E+18|#Cristiano You ne...|      en|  NEUTRAL|
| 2018-06-08T15:20:06|1.01E+18|#youngy18 #Englan...|      en| POSITIVE|
|2018-07-28T17:25:...|1.02E+18|#LFC #officialAL2...|      en|  NEUTRAL|
+--------------------+--------+--------------------+--------+---------+
only showing top 5 rows



In [5]:
print("Number of duplicates=",df.count()-df.drop_duplicates().count())

Number of duplicates= 1023944


Drop the duplicates

In [6]:
df=df.drop_duplicates()
count=df.count()
print(count)

6592620


In [7]:
df=df.drop('language')
df.show(5)

+--------------------+--------+--------------------+---------+
|  tweet_date_created|tweet_id|          tweet_text|sentiment|
+--------------------+--------+--------------------+---------+
|2018-07-20T19:46:...|1.02E+18|#leeabellis #andr...|  NEUTRAL|
|2018-09-12T20:32:...|1.04E+18|#Paschal_MUFC #LF...|  NEUTRAL|
| 2018-06-02T12:59:17|1.00E+18|#jnoahmorgan #hen...|  NEUTRAL|
|2018-07-24T08:30:...|1.02E+18|#HenrikhMkh #Auba...| POSITIVE|
|2018-07-10T22:47:...|1.02E+18|#Howmanyroads1 #b...|  NEUTRAL|
+--------------------+--------+--------------------+---------+
only showing top 5 rows



Clean date

In [8]:
from pyspark.sql.types import *
from pyspark.sql.functions import udf,expr,col
df=df.withColumn("tweet_date_created",col('tweet_date_created').cast('string'))
df.printSchema()

root
 |-- tweet_date_created: string (nullable = true)
 |-- tweet_id: string (nullable = true)
 |-- tweet_text: string (nullable = true)
 |-- sentiment: string (nullable = true)



In [9]:
def cleanDate(date):
  date = date.split("T")[0]
  return (date)
cleanDateFunction=udf(lambda x:cleanDate(x),StringType())
df=df.withColumn("tweet_date_created",cleanDateFunction(df.tweet_date_created))
df.show(5)

+------------------+--------+--------------------+---------+
|tweet_date_created|tweet_id|          tweet_text|sentiment|
+------------------+--------+--------------------+---------+
|        2018-07-20|1.02E+18|#leeabellis #andr...|  NEUTRAL|
|        2018-09-12|1.04E+18|#Paschal_MUFC #LF...|  NEUTRAL|
|        2018-06-02|1.00E+18|#jnoahmorgan #hen...|  NEUTRAL|
|        2018-07-24|1.02E+18|#HenrikhMkh #Auba...| POSITIVE|
|        2018-07-10|1.02E+18|#Howmanyroads1 #b...|  NEUTRAL|
+------------------+--------+--------------------+---------+
only showing top 5 rows



Remove retweets

In [10]:
print("Before=",count)

Before= 6592620


In [11]:
df=df.filter(~df.tweet_text.contains('Retweeted'))
df.show(5)

+------------------+--------+--------------------+---------+
|tweet_date_created|tweet_id|          tweet_text|sentiment|
+------------------+--------+--------------------+---------+
|        2018-07-20|1.02E+18|#leeabellis #andr...|  NEUTRAL|
|        2018-09-12|1.04E+18|#Paschal_MUFC #LF...|  NEUTRAL|
|        2018-06-02|1.00E+18|#jnoahmorgan #hen...|  NEUTRAL|
|        2018-07-24|1.02E+18|#HenrikhMkh #Auba...| POSITIVE|
|        2018-07-10|1.02E+18|#Howmanyroads1 #b...|  NEUTRAL|
+------------------+--------+--------------------+---------+
only showing top 5 rows



In [12]:
count=df.count()
print("After=",count)

After= 6590377


Remove irrelevent tweets

In [13]:
df=df.filter("Sentiment like '%POSITIVE%' or Sentiment like '%NEGATIVE%' or Sentiment like '%NEUTRAL%'")
df=df.filter(~df.tweet_text.contains('http'))
count=df.count()
print(count)

6234684


Realized you dont need tweet_Id, tweet_date as well

In [14]:
df=df.drop('tweet_date_created','tweet_id')
df.show(5)

+--------------------+---------+
|          tweet_text|sentiment|
+--------------------+---------+
|#leeabellis #andr...|  NEUTRAL|
|#Paschal_MUFC #LF...|  NEUTRAL|
|#jnoahmorgan #hen...|  NEUTRAL|
|#HenrikhMkh #Auba...| POSITIVE|
|#Howmanyroads1 #b...|  NEUTRAL|
+--------------------+---------+
only showing top 5 rows



Removing Stop words and tokenize

In [15]:
from pyspark.ml.feature import StopWordsRemover

def tokenizeText(text):
  lst=[]
  text=text.split(' ')
  for words in text:
    if words.find('@')==-1:
      if words.find('#')==-1:
        lst.append(words)
  return lst
tokenizeTextFunction=udf(lambda x: tokenizeText(x),ArrayType(StringType()))
df=df.withColumn("SplittedWords",tokenizeTextFunction(df.tweet_text))
df.show()

stopwordsremover=StopWordsRemover(inputCol="SplittedWords",outputCol="SplittedWordsNew")
df=stopwordsremover.transform(df)
df.show()

+--------------------+---------+--------------------+
|          tweet_text|sentiment|       SplittedWords|
+--------------------+---------+--------------------+
|#leeabellis #andr...|  NEUTRAL|[How, is, he, a, ...|
|#Paschal_MUFC #LF...|  NEUTRAL|[How, did, you, m...|
|#jnoahmorgan #hen...|  NEUTRAL|[I, just, apologi...|
|#HenrikhMkh #Auba...| POSITIVE|[Good, luck, this...|
|#Howmanyroads1 #b...|  NEUTRAL|[I, believe, he, ...|
|Every starter for...|  NEUTRAL|[Every, starter, ...|
|#chazaustin10 #CA...|  NEUTRAL|[Charkie10, ?...,...|
|#D_DeGea #BumperG...|  NEUTRAL|[Come, on, Baxter...|
|#MarkMoraghan #18...|  NEUTRAL|[I, will, fight, ...|
|#andresiniesta8 M...|  NEUTRAL|[Muchas, gracias,...|
|#GoloFutbol #MrDt...|  NEUTRAL|[At, least, Lacaz...|
|#PaulMJ875 #Cam_0...|  NEUTRAL|[considering, the...|
|#MarcusRashford #...| POSITIVE|[This, gave, me, ...|
|#SpursOfficial #A...|  NEUTRAL|[Hugo, certainly,...|
|#sheffraf #jamesd...|  NEUTRAL|[England, are, th...|
|China Goods #paul...| POSIT

In [16]:
df=df.drop('SplittedWords')

In [ ]:
!pip install nltk

stemming and lemmatization of data

In [17]:
from nltk.stem import SnowballStemmer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [18]:
stemmer = SnowballStemmer(language='english')
lemmatizer = WordNetLemmatizer()

def stem_words(words):
    return [stemmer.stem(word) for word in words]

def lemmatize_words(words):
    return [lemmatizer.lemmatize(word) for word in words]

stemming_udf = udf(stem_words, ArrayType(StringType()))
lemmatization_udf = udf(lemmatize_words, ArrayType(StringType()))

df = df.withColumn('stemmed_words', stemming_udf('SplittedWordsNew'))
df = df.withColumn('lemmatized_words', lemmatization_udf('stemmed_words'))

df.show()

+--------------------+---------+--------------------+--------------------+--------------------+
|          tweet_text|sentiment|    SplittedWordsNew|       stemmed_words|    lemmatized_words|
+--------------------+---------+--------------------+--------------------+--------------------+
|#leeabellis #andr...|  NEUTRAL|[big, loss?, bare...|[big, loss?, bare...|[big, loss?, bare...|
|#Paschal_MUFC #LF...|  NEUTRAL|[mean, country, b...|[mean, countri, b...|[mean, countri, b...|
|#jnoahmorgan #hen...|  NEUTRAL|[apologize, ?., I...|[apolog, ?., i ll...|[apolog, ?., i ll...|
|#HenrikhMkh #Auba...| POSITIVE|[Good, luck, seas...|[good, luck, seas...|[good, luck, seas...|
|#Howmanyroads1 #b...|  NEUTRAL|[believe, verball...|[believ, verbal, ...|[believ, verbal, ...|
|Every starter for...|  NEUTRAL|[Every, starter, ...|[everi, starter, ...|[everi, starter, ...|
|#chazaustin10 #CA...|  NEUTRAL|[Charkie10, ?...,...|[charkie10, ?...,...|[charkie10, ?...,...|
|#D_DeGea #BumperG...|  NEUTRAL|[Come, B

Handling emojis (i forgot bout that earlier)

In [19]:
import re

def removeEmojis(words):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return [emoji_pattern.sub(r'', word) for word in words]

removeEmojisFunction = udf(removeEmojis, ArrayType(StringType()))

df = df.withColumn('no_emojis', removeEmojisFunction(df.lemmatized_words))
df.show(5)

+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|          tweet_text|sentiment|    SplittedWordsNew|       stemmed_words|    lemmatized_words|           no_emojis|
+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|#leeabellis #andr...|  NEUTRAL|[big, loss?, bare...|[big, loss?, bare...|[big, loss?, bare...|[big, loss?, bare...|
|#Paschal_MUFC #LF...|  NEUTRAL|[mean, country, b...|[mean, countri, b...|[mean, countri, b...|[mean, countri, b...|
|#jnoahmorgan #hen...|  NEUTRAL|[apologize, ?., I...|[apolog, ?., i ll...|[apolog, ?., i ll...|[apolog, ?., ill,...|
|#HenrikhMkh #Auba...| POSITIVE|[Good, luck, seas...|[good, luck, seas...|[good, luck, seas...|[good, luck, seas...|
|#Howmanyroads1 #b...|  NEUTRAL|[believe, verball...|[believ, verbal, ...|[believ, verbal, ...|[believ, verbal, ...|
+--------------------+---------+--------------------+-----------

#Sentiment Analysis

Using pandas dataframe

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
pandas_df=df.toPandas()

pandas_df['no_emojis'] = pandas_df['no_emojis'].apply(' '.join)
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(pandas_df['no_emojis'])
X_train, X_test, y_train, y_test = train_test_split(X, pandas_df['sentiment'], test_size=0.2, random_state=42)
lr = LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("accuracy=",accuracy)

accuracy= 0.7785016286644951


Using pyspark ML

In [34]:
from pyspark.ml.feature import CountVectorizer, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
df=df.drop('features','label')

indexer = StringIndexer(inputCol='sentiment', outputCol='label')
df = indexer.fit(df).transform(df)
vectorizer = CountVectorizer(inputCol='no_emojis', outputCol='features')
model = vectorizer.fit(df)
df = model.transform(df)

train, test = df.randomSplit([0.8, 0.2], seed=42)


lr = LogisticRegression(labelCol='label')
lr_model = lr.fit(train)

predictions = lr_model.transform(test)

evaluator = MulticlassClassificationEvaluator(labelCol='label', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print("accuracy=",accuracy)
df=df.drop('features','label')

accuracy= 0.7038461538461539


Using nltk

In [35]:
from nltk.sentiment import SentimentIntensityAnalyzer
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

def computeSentiment(words):
    text = ' '.join(words)
    sentiment_scores = sia.polarity_scores(text)
    if sentiment_scores['compound'] > 0:
        return 'positive'
    elif sentiment_scores['compound'] < 0:
        return 'negative'
    else:
        return 'neutral'

computeSentimentFunction = udf(computeSentiment, StringType())
df = df.withColumn('SentimentResult', computeSentimentFunction(df['no_emojis']))
df.show()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


+--------------------+---------+--------------------+--------------------+--------------------+--------------------+---------------+
|          tweet_text|sentiment|    SplittedWordsNew|       stemmed_words|    lemmatized_words|           no_emojis|SentimentResult|
+--------------------+---------+--------------------+--------------------+--------------------+--------------------+---------------+
|#leeabellis #andr...|  NEUTRAL|[big, loss?, bare...|[big, loss?, bare...|[big, loss?, bare...|[big, loss?, bare...|       positive|
|#Paschal_MUFC #LF...|  NEUTRAL|[mean, country, b...|[mean, countri, b...|[mean, countri, b...|[mean, countri, b...|       negative|
|#jnoahmorgan #hen...|  NEUTRAL|[apologize, ?., I...|[apolog, ?., i ll...|[apolog, ?., i ll...|[apolog, ?., ill,...|       negative|
|#HenrikhMkh #Auba...| POSITIVE|[Good, luck, seas...|[good, luck, seas...|[good, luck, seas...|[good, luck, seas...|       positive|
|#Howmanyroads1 #b...|  NEUTRAL|[believe, verball...|[believ, verbal,

In [36]:
confusion_matrix = df.crosstab('sentiment', 'SentimentResult')
confusion_matrix.show()

+-------------------------+--------+-------+--------+
|sentiment_SentimentResult|negative|neutral|positive|
+-------------------------+--------+-------+--------+
|                 POSITIVE|   50070| 100141|  518305|
|                 NEGATIVE|  229538|  45001|   70457|
|                  NEUTRAL|  393905|1945364|  242446|
+-------------------------+--------+-------+--------+



In [37]:
df.groupby('Sentiment').count().show()

+---------+--------+
|Sentiment|   count|
+---------+--------+
| POSITIVE|  668516|
| NEGATIVE|  264996|
|  NEUTRAL| 2502936|
+---------+--------+

